<a href="https://colab.research.google.com/github/javadvahid/deepLearning/blob/main/CryptoPricePredictionPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#read data download from cryptodatadownload.com
data = pd.read_csv("/content/Bitfinex_BTCUSD_d.csv")

In [ ]:
data.head()


,unix,date,symbol,open,high,low,close,Volume USD,Volume BTC
0,1.625440e+12,7/5/2021 0:00,BTC/USD,35278.00000,35280.0,34851.0,34913.00000,8.843958e+06,253.314189
1,1.625360e+12,7/4/2021 0:00,BTC/USD,34689.00000,35950.0,34376.0,35276.00000,1.033442e+08,2929.589307
2,1.625270e+12,7/3/2021 0:00,BTC/USD,33818.00000,34948.0,33350.0,34682.00000,1.028887e+08,2966.629686
3,1.625180e+12,7/2/2021 0:00,BTC/USD,33519.46745,33983.0,32700.0,33816.00000,1.739982e+08,5145.439921
4,1.625100e+12,7/1/2021 0:00,BTC/USD,35062.00000,35069.0,32720.0,33554.42277,1.920285e+08,5722.895795


In [ ]:
#we only use the Close price of each day
dataPrice = data.close

In [ ]:
#preparing train and test data
dp_train = dataPrice[:1872].values # 80% of 2339 for train
dp_test = dataPrice[1872:].values # 20% of 2339 for test

In [ ]:
#scaling data to the range of [0 1]

#finding the Min and the Max of data
scaler = MinMaxScaler().fit(dataPrice.values.reshape(-1,1)) 
#transforming data using the Max and Min to the range of [0 1]. you can also do that in 1 step using MinMaxScaler().fit_transform(data)
dp_train = scaler.transform(dp_train.reshape(-1,1)) 
print(dp_train.shape)
#do the same for test dataset
dp_test = scaler.transform(dp_test.reshape(-1,1))


#creating X and Y for Train and Test
nod = 20 #Number Of Days used for predicting the next day's price
x_train = []
y_train = []
x_test = []
y_test = []
for i in range (len(dp_train) - nod):
  x_train.append(dp_train[i:(i+nod)])
  y_train.append(dp_train[i + nod])

for j in range (len(dp_test) - nod):
  x_test.append(dp_test[j:(j+nod)])
  y_test.append(dp_test[j + nod])  

batch_size = 1

# LSTM Layer in Pytorch get an input of size (L, N, H) 
x_train = torch.from_numpy(np.array(x_train).squeeze().astype(np.float32)) 
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))

(1872, 1)


In [ ]:
print(x_train.size())

torch.Size([1852, 20])


In [ ]:
#model
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.lstm1 = nn.LSTM(input_size = 20, hidden_size = 10)
    self.linear = nn.Linear(10, 1)

  def forward(self, x):
    xbatch = x.view(len(x), 1, -1) 
    x, c = self.lstm1(xbatch)
    x = self.linear(x)
    return x  

In [ ]:
model = Model()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
print(model)

Model(
  (lstm1): LSTM(20, 10)
  (linear): Linear(in_features=10, out_features=1, bias=True)
)


In [ ]:
#train
epochs = 1000
for i in range(epochs):
  model.zero_grad()
  output = model(x_train)
  loss = criterion(output, y_train)
  if i % 20 == 0:
        print("epoch {}   loss: {} ".format(i, loss.item()))
  loss.backward()
  optimizer.step()


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1852, 1])) that is different to the input size (torch.Size([1852, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0   loss: 0.04337636008858681 
epoch 20   loss: 0.013367475010454655 
epoch 40   loss: 0.0019406317733228207 
epoch 60   loss: 1.2616384083230514e-05 
epoch 80   loss: 4.399312092573382e-05 
epoch 100   loss: 1.234092178492574e-05 
epoch 120   loss: 1.2573450476338621e-05 
epoch 140   loss: 1.2161594895587768e-05 
epoch 160   loss: 1.2122308362449985e-05 
epoch 180   loss: 1.210849586641416e-05 
epoch 200   loss: 1.2093448276573326e-05 
epoch 220   loss: 1.207856348628411e-05 
epoch 240   loss: 1.2063001122442074e-05 
epoch 260   loss: 1.2046723895764444e-05 
epoch 280   loss: 1.2029814570269082e-05 
epoch 300   loss: 1.2012234947178513e-05 
epoch 320   loss: 1.1994072337984107e-05 
epoch 340   loss: 1.1975299457844812e-05 
epoch 360   loss: 1.1955936315644067e-05 
epoch 380   loss: 1.1936011105717625e-05 
epoch 400   loss: 1.191555020341184e-05 
epoch 420   loss: 1.1894534509337973e-05 
epoch 440   loss: 1.1873029507114552e-05 
epoch 460   loss: 1.1850998816953506e-05 
epoch 480

In [ ]:
# test

price = model(x_test)